In [1]:
# 1) Imports

# pandas: import and store data - numpy just in case
import pandas as pd
import numpy as np

# bokeh for interactive plotting
from bokeh.models import BoxZoomTool
from bokeh.plotting import figure, output_notebook, show

# bokeh for the map tiles
from bokeh.tile_providers import STAMEN_TERRAIN

# datashader stuff
import datashader as ds
import datashader.transfer_functions as tf
from datashader.bokeh_ext import InteractiveImage
from datashader.utils import export_image
from datashader.colors import colormap_select, Greys9, Hot, viridis, inferno  # do we need these?

from pyproj import Proj, transform

output_notebook()



C:\Users\Taylor\AppData\Local\conda\conda\envs\ds\lib\site-packages\odo\backends\pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))


Loading BokehJS ...

In [2]:
print(type(STAMEN_TERRAIN))

<class 'bokeh.models.tiles.WMTSTileSource'>


In [3]:
# df = pd.read_csv("../data/daily_agg/all_daily_2017-07-30.csv", skipinitialspace=True)
df = pd.read_csv("../data/daily_agg/all_daily_2017-07-30_clean.csv", skipinitialspace=True)

C:\Users\Taylor\AppData\Local\conda\conda\envs\ds\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# https://gis.stackexchange.com/questions/78838/converting-projected-coordinates-to-lat-lon-using-python
inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:3857' )
# x1, y1 = -105.150271116, 39.7278572773
# x1,y1 = -11705274.6374,4826473.6922
# x2,y2 = transform(inProj,outProj,x1,y1)

df['Lat2'], df['Long2'] = transform(inProj, outProj, df['Lat'].tolist(), df['Long'].tolist())
print(df.head())

   Id    Icao   Rcvr        Lat       Long  Reg  Fseen  Tsecs  Cmsgs      Alt  \
0   1  000001  21021  35.362289 -78.238220  NaN    NaN    NaN    NaN   7600.0   
1   1  000001  11085  35.450317 -78.182888  NaN    NaN    NaN    NaN   9725.0   
2   1  000001  21021  35.450317 -78.182888  NaN    NaN    NaN    NaN  10100.0   
3   1  000001  31027  35.450320 -78.182918  NaN    NaN    NaN    NaN  10100.0   
4   1  000001  11085  35.529599 -78.125591  NaN    NaN    NaN    NaN  12700.0   

       ...       EngType    Mil                            Cou    Call  Talt  \
0      ...             0  False  Unknown or unassigned country  CLAW41   NaN   
1      ...             0  False  Unknown or unassigned country  CLAW41   NaN   
2      ...             0  False  Unknown or unassigned country  CLAW41   NaN   
3      ...             0  False  Unknown or unassigned country  CLAW41   NaN   
4      ...             0  False  Unknown or unassigned country  CLAW41   NaN   

   Ttrk          jsonfile  order

In [5]:
# 2) Function definitions

plot_width  = int(900)
plot_height = int(650)
# x_range, y_range = ((-10000,10000), (-10000,10000))

x_range, y_range = ((-8242000,-8210000), (4965000,4990000))

# create the bokeh interactive base plot
def base_plot(tools='pan,wheel_zoom,reset',plot_width=plot_width, plot_height=plot_height, **plot_args):
    p = figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
        x_range=x_range, y_range=y_range, outline_line_color=None,
        min_border=0, min_border_left=0, min_border_right=0,
        min_border_top=0, min_border_bottom=0, **plot_args)
    
    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    
    p.add_tools(BoxZoomTool(match_aspect=True))
    
    return p


# create the actual datashader plot to be layered over the bokeh figure
def create_image90(x_range, y_range, w=plot_width, h=plot_height):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    # agg = cvs.points(df, 'dropoff_x', 'dropoff_y',  ds.count('passenger_count'))
    agg = cvs.points(df, 'Long2', 'Lat2', ds.count('Id'))
    # img = tf.shade(agg.where(agg>np.percentile(agg,90)), cmap=inferno, how='eq_hist')
    # img = tf.shade(agg, cmap=inferno)
    img = tf.shade(agg, cmap=inferno, how='log')
    return tf.dynspread(img, threshold=0.3, max_px=4)


In [6]:
p = base_plot()
p.add_tile(STAMEN_TERRAIN)
InteractiveImage(p, create_image90)